In [29]:
import pandas as pd
import os

In [41]:
df = pd.DataFrame()
files = os.listdir("./patients_new")
for file in files:
    patient = pd.read_json(os.path.join("./patients_new/", file))
    df = pd.concat([df, patient], ignore_index=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   subject_id            275 non-null    int64 
 1   hadm_id               275 non-null    int64 
 2   admittime             275 non-null    object
 3   dischtime             275 non-null    object
 4   deathtime             15 non-null     object
 5   admission_type        275 non-null    object
 6   admit_provider_id     275 non-null    object
 7   admission_location    275 non-null    object
 8   discharge_location    233 non-null    object
 9   insurance             275 non-null    object
 10  language              275 non-null    object
 11  marital_status        263 non-null    object
 12  race                  275 non-null    object
 13  edregtime             182 non-null    object
 14  edouttime             182 non-null    object
 15  hospital_expire_flag  275 non-null    in

In [ ]:
df.info()

In [42]:
df

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,...,hcpcsevents,labevents,microbiologyevents,pharmacy,poe,prescriptions,procedures_icd,services,transfers,icustays
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,...,[],"[{'labevent_id': 151, 'subject_id': 10000032, ...","[{'microevent_id': 15, 'subject_id': 10000032,...","[{'subject_id': 10000032, 'hadm_id': 22595853,...","[{'poe_id': '10000032-52', 'poe_seq': 52, 'sub...","[{'subject_id': 10000032, 'hadm_id': 22595853,...","[{'subject_id': 10000032, 'hadm_id': 22595853,...","[{'subject_id': 10000032, 'hadm_id': 22595853,...","[{'subject_id': 10000032, 'hadm_id': 22595853....",[]
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,...,[],"[{'labevent_id': 332, 'subject_id': 10000032, ...","[{'microevent_id': 28, 'subject_id': 10000032,...","[{'subject_id': 10000032, 'hadm_id': 22841357,...","[{'poe_id': '10000032-69', 'poe_seq': 69, 'sub...","[{'subject_id': 10000032, 'hadm_id': 22841357,...","[{'subject_id': 10000032, 'hadm_id': 22841357,...","[{'subject_id': 10000032, 'hadm_id': 22841357,...","[{'subject_id': 10000032, 'hadm_id': 22841357....",[]
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,...,[],"[{'labevent_id': 584, 'subject_id': 10000032, ...","[{'microevent_id': 36, 'subject_id': 10000032,...","[{'subject_id': 10000032, 'hadm_id': 25742920,...","[{'poe_id': '10000032-217', 'poe_seq': 217, 's...","[{'subject_id': 10000032, 'hadm_id': 25742920,...","[{'subject_id': 10000032, 'hadm_id': 25742920,...","[{'subject_id': 10000032, 'hadm_id': 25742920,...","[{'subject_id': 10000032, 'hadm_id': 25742920....",[]
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,...,[],"[{'labevent_id': 407, 'subject_id': 10000032, ...","[{'microevent_id': 32, 'subject_id': 10000032,...","[{'subject_id': 10000032, 'hadm_id': 29079034,...","[{'poe_id': '10000032-125', 'poe_seq': 125, 's...","[{'subject_id': 10000032, 'hadm_id': 29079034,...",[],"[{'subject_id': 10000032, 'hadm_id': 29079034,...","[{'subject_id': 10000032, 'hadm_id': 29079034....","[{'subject_id': 10000032, 'hadm_id': 29079034,..."
4,10001217,24597018,2157-11-18 22:56:00,2157-11-25 18:00:00,NaN,EW EMER.,P4645A,EMERGENCY ROOM,HOME HEALTH CARE,Other,...,[],"[{'labevent_id': 9011, 'subject_id': 10001217,...","[{'microevent_id': 402, 'subject_id': 10001217...","[{'subject_id': 10001217, 'hadm_id': 24597018,...","[{'poe_id': '10001217-110', 'poe_seq': 110, 's...","[{'subject_id': 10001217, 'hadm_id': 24597018,...","[{'subject_id': 10001217, 'hadm_id': 24597018,...","[{'subject_id': 10001217, 'hadm_id': 24597018,...","[{'subject_id': 10001217, 'hadm_id': 24597018....","[{'subject_id': 10001217, 'hadm_id': 24597018,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,10040025,21791856,2147-06-16 22:00:00,2147-06-22 16:15:00,NaN,OBSERVATION ADMIT,P7634X,EMERGENCY ROOM,HOME HEALTH CARE,Other,...,[],"[{'labevent_id': 472612, 'subject_id': 1004002...","[{'microevent_id': 14525, 'subject_id': 100400...","[{'subject_id': 10040025, 'hadm_id': 21791856,...","[{'poe_id': '10040025-451', 'poe_seq': 451, 's...","[{'subject_id': 10040025, 'hadm_id': 21791856,...",[],"[{'subject_id': 10040025, 'hadm_id': 21791856,...","[{'subject_id': 10040025, 'hadm_id': 21791856....",[]
271,10040025,22251969,2147-08-03 02:58:00,2147-08-06 16:50:00,NaN,OBSERVATION ADMIT,P85BWS,EMERGENCY ROOM,HOME,Other,...,[],"[{'labevent_id': 472861, 'subject_id': 1004002...",[],"[{'subject_id': 10040025, 'hadm_id': 22251969,...","[{'poe_id': '10040025-594', 'poe_seq': 594, 's...","[{'subject_id': 10040025, 'hadm_id': 22251969,...",[],"[{'subject_id': 10040025, 'hadm_id': 22251969,...","[{'subject_id': 10040025, 'had

In [ ]:
omr = pd.DataFrame(df.loc[1, "omr"]).sort_values(by=["result_name", "chartdate"]).drop_duplicates(subset=['subject_id', 'chartdate', 'result_name', 'result_value'])
omr

In [ ]:
emar = pd.DataFrame(df.loc[1, "emar"]).sort_values(by=["charttime"])
emar

In [ ]:
patients = pd.DataFrame(df.loc[1, "patients"])
patients